# FFNN

In [20]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import r2_score as sklearn_r2_score


# Load and preprocess the data
data_directory = '../../Data/Filtered_split_training_data/'
chromosome_number = 1

hidden_size1 = 150
hidden_size2 = 150

for chromosome_number in range (1,23):
    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_split.parquet"
    data = pd.read_parquet(file_name)
    print("Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Unknown" in col]].shape[1])
    print("Known PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Known" in col]].shape[1])
    print("23AndMe SNPs with LD to Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_" not in col]].shape[1])

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*Unknown)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='Unknown').values, dtype=torch.float32)

        
        # Define the updated neural network architecture
    class SimpleFFNN(nn.Module):
        def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
            super(SimpleFFNN, self).__init__()
            self.hidden1 = nn.Linear(input_size, hidden_size1)
            self.hidden2 = nn.Linear(hidden_size1, hidden_size2)
            self.output = nn.Linear(hidden_size2, output_size)

        def forward(self, x):
            x = torch.relu(self.hidden1(x))
            x = torch.relu(self.hidden2(x))
            x = torch.sigmoid(self.output(x))
            return x

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters
    input_dim = X.shape[1]
    output_dim = y.shape[1]
    learning_rate = 0.001
    num_epochs = 400
    batch_size = 128
    num_folds = 5

    # Define the loss function
    criterion = nn.BCELoss()

    # K-fold cross-validation
    kfold = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    fold_accuracies = []
    fold_precisions = []
    fold_recalls = []
    fold_f1_scores = []
    fold_roc_auc_scores = []
    fold_r2_scores = []
    fold_train_losses = []
    fold_val_losses = []

    for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
        print(f"Fold {fold + 1}/{num_folds}")
        
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        val_dataset = TensorDataset(X_val, y_val)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)
        
        model = SimpleFFNN(input_dim,hidden_size1, hidden_size2, output_dim).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        
        for epoch in range(num_epochs):
            train_loss = 0.0
            for batch_X, batch_y in train_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                
                train_loss += loss.item()
            
            train_loss /= len(train_loader)
            fold_train_losses.append(train_loss)
        
        with torch.no_grad():
            val_outputs = model(X_val.to(device))
            val_loss = criterion(val_outputs, y_val.to(device))
            fold_val_losses.append(val_loss.item())
            
            val_preds = (val_outputs > 0.5).float()
            val_accuracy = float(((val_preds > 0.5) == y_val).float().mean())
            val_precision = precision_score(y_val.cpu().numpy(), val_preds.cpu().numpy(), average='micro')
            val_recall = recall_score(y_val.cpu().numpy(), val_preds.cpu().numpy(), average='micro')
            val_f1 = f1_score(y_val.cpu().numpy(), val_preds.cpu().numpy(), average='micro')
            val_roc_auc = roc_auc_score(y_val.cpu().numpy(), val_outputs.cpu().numpy(), average='micro')
            val_r2 = sklearn_r2_score(y_val.cpu().numpy(), val_outputs.cpu().numpy())

            fold_accuracies.append(val_accuracy)
            fold_precisions.append(val_precision)
            fold_recalls.append(val_recall)
            fold_f1_scores.append(val_f1)
            fold_roc_auc_scores.append(val_roc_auc)
            fold_r2_scores.append(val_r2)

            print(f"Fold {fold + 1}/{num_folds}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss.item():.4f}, Val Accuracy: {val_accuracy:.4f}, Val Precision: {val_precision:.4f}, Val Recall: {val_recall:.4f}, Val F1: {val_f1:.4f}, Val ROC AUC: {val_roc_auc:.4f}, Val R2: {val_r2:.4f}")


    print(f"Average Accuracy: {np.mean(fold_accuracies):.4f} +/- {np.std(fold_accuracies):.4f}")
    print(f"Average Precision: {np.mean(fold_precisions):.4f} +/- {np.std(fold_precisions):.4f}")
    print(f"Average Recall: {np.mean(fold_recalls):.4f} +/- {np.std(fold_recalls):.4f}")
    print(f"Average F1 Score: {np.mean(fold_f1_scores):.4f} +/- {np.std(fold_f1_scores):.4f}")
    print(f"Average ROC AUC: {np.mean(fold_roc_auc_scores):.4f} +/- {np.std(fold_roc_auc_scores):.4f}")
    print(f"Average R2 Score: {np.mean(fold_r2_scores):.4f} +/- {np.std(fold_r2_scores):.4f}")

    import csv

    # Export results to CSV

    output_folder = "../../Data/model_results/logistic_regression/"

    csv_file = output_folder + f'cross_validation_results_chr{chromosome_number}.csv'
    fieldnames = ['Fold', 'Train Loss', 'Val Loss', 'Val Accuracy', 'Val Precision', 'Val Recall', 'Val F1', 'Val ROC AUC', 'Val R2']

    with open(csv_file, 'w', newline='') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames)
        writer.writeheader()

        for fold in range(num_folds):
            writer.writerow({
                'Fold': fold + 1,
                'Train Loss': fold_train_losses[fold],
                'Val Loss': fold_val_losses[fold],
                'Val Accuracy': fold_accuracies[fold],
                'Val Precision': fold_precisions[fold],
                'Val Recall': fold_recalls[fold],
                'Val F1': fold_f1_scores[fold],
                'Val ROC AUC': fold_roc_auc_scores[fold],
                'Val R2': fold_r2_scores[fold]
            })

        writer.writerow({})  # Empty row for separation
        writer.writerow({
            'Fold': 'Average',
            'Train Loss': np.mean(fold_train_losses),
            'Val Loss': np.mean(fold_val_losses),
            'Val Accuracy': np.mean(fold_accuracies),
            'Val Precision': np.mean(fold_precisions),
            'Val Recall': np.mean(fold_recalls),
            'Val F1': np.mean(fold_f1_scores),
            'Val ROC AUC': np.mean(fold_roc_auc_scores),
            'Val R2': np.mean(fold_r2_scores)
        })
        writer.writerow({
            'Fold': 'Std Dev',
            'Train Loss': np.std(fold_train_losses),
            'Val Loss': np.std(fold_val_losses),
            'Val Accuracy': np.std(fold_accuracies),
            'Val Precision': np.std(fold_precisions),
            'Val Recall': np.std(fold_recalls),
            'Val F1': np.std(fold_f1_scores),
            'Val ROC AUC': np.std(fold_roc_auc_scores),
            'Val R2': np.std(fold_r2_scores)
        })

    print(f"Results exported to {csv_file}")

Unknown PRS313 SNPs:  40
Known PRS313 SNPs:  20
23AndMe SNPs with LD to Unknown PRS313 SNPs:  2204
Fold 1/5
Fold 1/5, Train Loss: 0.0000, Val Loss: 0.8972, Val Accuracy: 0.9076, Val Precision: 0.8658, Val Recall: 0.8268, Val F1: 0.8458, Val ROC AUC: 0.9522, Val R2: 0.5074
Fold 2/5


KeyboardInterrupt: 

In [8]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.tensorboard import SummaryWriter
import os

# Define the updated neural network architecture
class SimpleFFNN(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, output_size):
        super(SimpleFFNN, self).__init__()
        self.hidden1 = nn.Linear(input_size, hidden_size1)
        self.hidden2 = nn.Linear(hidden_size1, hidden_size2)
        self.output = nn.Linear(hidden_size2, output_size)

    def forward(self, x):
        x = torch.relu(self.hidden1(x))
        x = torch.relu(self.hidden2(x))
        x = torch.sigmoid(self.output(x))
        return x

def train_and_evaluate_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, device, writer, checkpoint_path):
    best_val_loss = float('inf')
    start_epoch = 0

    # Load checkpoint if it exists
    if os.path.exists(checkpoint_path):
        checkpoint = torch.load(checkpoint_path)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        best_val_loss = checkpoint['best_val_loss']
        print(f"Loaded checkpoint from {checkpoint_path}. Starting from epoch {start_epoch + 1}")

    for epoch in range(start_epoch, num_epochs):
        model.train()
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_loss = 0
            val_outputs = []
            val_labels = []
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                outputs = model(batch_X)
                val_loss += criterion(outputs, batch_y).item()
                val_outputs.extend(outputs.cpu().numpy())
                val_labels.extend(batch_y.cpu().numpy())
            
            val_loss /= len(val_loader)
            scheduler.step(val_loss)  # Update the learning rate based on validation loss
            val_outputs = np.array(val_outputs)
            val_labels = np.array(val_labels)
            val_accuracy = ((val_outputs > 0.5) == val_labels).mean()
            # val_accuracy = accuracy_score((val_labels == 1), np.round(val_outputs))
            # val_roc_auc = roc_auc_score(val_labels, val_outputs, average="macro")
            
            writer.add_scalar('Val_Loss', val_loss, epoch)
            writer.add_scalar('Val_Accuracy', val_accuracy, epoch)
            # writer.add_scalar('Val_ROC_AUC', val_roc_auc, epoch)
            
            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'best_val_loss': best_val_loss
                }, checkpoint_path)
                print("Validation Accuracy is ", val_accuracy)
                print(f"Epoch {epoch+1}: Val Loss improved to {val_loss:.4f}, saved checkpoint")
            else:
                print(f"Epoch {epoch+1}: Val Loss did not improve from {best_val_loss:.4f}")
    
    return best_val_loss

# Load and preprocess the data
data_directory = '../../Data/Filtered_split_training_data/'
chromosome_number = 13
file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_split.parquet"
data = pd.read_parquet(file_name)

print("Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Unknown" in col]].shape[1])
print("Known PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Known" in col]].shape[1])
print("23AndMe SNPs with LD to Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_" not in col]].shape[1])

# Split the data into train and validation sets
X = torch.tensor(data.filter(regex='^(?!.*Unknown)').values, dtype=torch.float32)
y = torch.tensor(data.filter(regex='Unknown').values, dtype=torch.float32)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)

val_dataset = TensorDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=128)

# Set up the model, loss function, optimizer, and learning rate scheduler
input_size = X_train.shape[1]
hidden_size1 = 100
hidden_size2 = 100
output_size = y_train.shape[1]

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleFFNN(input_size, hidden_size1, hidden_size2, output_size).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.01, patience=5, verbose=True)

# Train the model
num_epochs = 400
checkpoint_path = f'checkpoint_chr{chromosome_number}.pth'
writer = SummaryWriter()
best_val_loss = train_and_evaluate_model(model, criterion, optimizer, scheduler, train_loader, val_loader, num_epochs, device, writer, checkpoint_path)
writer.close()

# Load the best model and evaluate on the validation set
best_model = SimpleFFNN(input_size, hidden_size1, hidden_size2, output_size).to(device)
checkpoint = torch.load(checkpoint_path)
best_model.load_state_dict(checkpoint['model_state_dict'])
best_model.eval()

with torch.no_grad():
    val_accuracies = []
    
    for batch_X, batch_y in val_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = best_model(batch_X)
        val_accuracy = ((outputs > 0.5) == batch_y.to(device)).float().mean()

        val_accuracies.append(val_accuracy.cpu())

    print(np.mean(val_accuracies))
    # val_roc_auc = roc_auc_score(val_labels, val_outputs, average="macro")
    
    print(f"Best Model - Val Accuracy: {val_accuracy:.4f}")

Unknown PRS313 SNPs:  8
Known PRS313 SNPs:  2
23AndMe SNPs with LD to Unknown PRS313 SNPs:  210
Loaded checkpoint from checkpoint_chr13.pth. Starting from epoch 44
Epoch 44: Val Loss did not improve from 0.1294
Epoch 45: Val Loss did not improve from 0.1294
Epoch 46: Val Loss did not improve from 0.1294
Epoch 47: Val Loss did not improve from 0.1294
Epoch 48: Val Loss did not improve from 0.1294
Epoch 49: Val Loss did not improve from 0.1294
Epoch 00007: reducing learning rate of group 0 to 1.0000e-05.
Epoch 50: Val Loss did not improve from 0.1294
Epoch 51: Val Loss did not improve from 0.1294
Epoch 52: Val Loss did not improve from 0.1294
Epoch 53: Val Loss did not improve from 0.1294
Epoch 54: Val Loss did not improve from 0.1294
Epoch 55: Val Loss did not improve from 0.1294
Epoch 56: Val Loss did not improve from 0.1294
Epoch 57: Val Loss did not improve from 0.1294
Epoch 00015: reducing learning rate of group 0 to 1.0000e-07.
Epoch 58: Val Loss did not improve from 0.1294
Epoch 5

KeyboardInterrupt: 

In [37]:
data.columns.to_list()

['chr22_46255004_A_G_maternal',
 'chr22_46255004_A_G_paternal',
 'chr22_46391695_C_T_maternal',
 'chr22_46391695_C_T_paternal',
 'chr22_46320275_A_G_maternal',
 'chr22_46320275_A_G_paternal',
 'chr22_46194973_C_T_maternal',
 'chr22_46194973_C_T_paternal',
 'chr22_46290571_T_C_maternal',
 'chr22_46290571_T_C_paternal',
 'chr22_46610409_A_G_maternal',
 'chr22_46610409_A_G_paternal',
 'chr22_46251927_A_G_maternal',
 'chr22_46251927_A_G_paternal',
 'chr22_46324957_T_C_maternal',
 'chr22_46324957_T_C_paternal',
 'chr22_46391888_A_G_maternal',
 'chr22_46391888_A_G_paternal',
 'chr22_46487902_A_G_maternal',
 'chr22_46487902_A_G_paternal',
 'chr22_46009063_A_G_maternal',
 'chr22_46009063_A_G_paternal',
 'chr22_46428306_T_C_maternal',
 'chr22_46428306_T_C_paternal',
 'chr22_46298649_G_A_maternal',
 'chr22_46298649_G_A_paternal',
 'chr22_46458123_T_G_maternal',
 'chr22_46458123_T_G_paternal',
 'chr22_46731689_G_T_maternal',
 'chr22_46731689_G_T_paternal',
 'chr22_46630634_G_C_maternal',
 'chr22_

In [32]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, roc_curve
from sklearn.metrics import r2_score as sklearn_r2_score
import optuna
from matplotlib import pyplot as plt

# Load and preprocess the data
data_directory = '../../Data/Filtered_split_training_data/'
start = 1

# Initialize lists to store the performance metrics for each chromosome
accuracies = []
precisions = []
recalls = []
false_positive_rates = []
auc_rocs = []
r2_scores = []


# Create folders for saving files
output_folder = "../../Data/model_results/logistic_regression/"
model_folder = output_folder + "models/"
csv_folder = output_folder + "csv_files/"
curve_folder = output_folder + "roc_curves/"

os.makedirs(model_folder, exist_ok=True)
os.makedirs(csv_folder, exist_ok=True)
os.makedirs(curve_folder, exist_ok=True)

for chromosome_number in range(start, 23):

    # Create subfolders for the current chromosome
    chr_model_folder = model_folder + f"chr{chromosome_number}/"
    chr_csv_folder = csv_folder + f"chr{chromosome_number}/"
    chr_curve_folder = curve_folder + f"chr{chromosome_number}/"

    os.makedirs(chr_model_folder, exist_ok=True)
    os.makedirs(chr_csv_folder, exist_ok=True)
    os.makedirs(chr_curve_folder, exist_ok=True)

    file_name = data_directory + f"23AndMe_PRS313_merged_chr{chromosome_number}_matching_split.parquet"
    data = pd.read_parquet(file_name)
    print("Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Unknown" in col]].shape[1])
    print("Known PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_Known" in col]].shape[1])
    print("23AndMe SNPs with LD to Unknown PRS313 SNPs: ", data[[col for col in data.columns if "PRS313_" not in col]].shape[1])

    # Split the data into features and target
    X = torch.tensor(data.filter(regex='^(?!.*Unknown)').values, dtype=torch.float32)
    y = torch.tensor(data.filter(regex='Unknown').values, dtype=torch.float32)

    # Split the data into train-validation and test sets
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Define the logistic regression model with lasso regularization
    class LogisticRegression(nn.Module):
        def __init__(self, input_dim, output_dim, lasso_coef=0.0):
            super(LogisticRegression, self).__init__()
            self.linear = nn.Linear(input_dim, output_dim)
            self.sigmoid = nn.Sigmoid()
            self.lasso_coef = lasso_coef

        def forward(self, x):
            out = self.linear(x)
            out = self.sigmoid(out)
            return out

        def lasso_loss(self):
            return self.lasso_coef * torch.norm(self.linear.weight, p=1)

    # Set the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Set the hyperparameters for tuning
    input_dim = X_train_val.shape[1]
    output_dim = y_train_val.shape[1]
    num_epochs = 500
    batch_size = 128

    # Define the objective function for Optuna with cross-validation and early stopping
    def objective(trial):
        learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
        lasso_coef = trial.suggest_float('lasso_coef', 1e-5, 1e-1, log=True)
        patience = trial.suggest_int('patience', 5, 20)

        model = LogisticRegression(input_dim, output_dim, lasso_coef).to(device)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
        criterion = nn.BCELoss()

        kfold = KFold(n_splits=5, shuffle=True, random_state=42)
        fold_losses = []

        for fold, (train_idx, val_idx) in enumerate(kfold.split(X_train_val)):
            X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]
            y_train, y_val = y_train_val[train_idx], y_train_val[val_idx]

            train_dataset = TensorDataset(X_train, y_train)
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

            best_val_loss = float('inf')
            counter = 0

            for epoch in range(num_epochs):
                train_loss = 0.0
                for batch_X, batch_y in train_loader:
                    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

                    outputs = model(batch_X)
                    loss = criterion(outputs, batch_y) + model.lasso_loss()

                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()

                    train_loss += loss.item()

                train_loss /= len(train_loader)

                val_dataset = TensorDataset(X_val, y_val)
                val_loader = DataLoader(val_dataset, batch_size=batch_size)

                with torch.no_grad():
                    val_loss = 0.0
                    for batch_X, batch_y in val_loader:
                        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
                        outputs = model(batch_X)
                        loss = criterion(outputs, batch_y) + model.lasso_loss()
                        val_loss += loss.item()

                    val_loss /= len(val_loader)

                    if val_loss < best_val_loss:
                        best_val_loss = val_loss
                        counter = 0
                    else:
                        counter += 1

                    if counter >= patience:
                        print(f"Early stopping at epoch {epoch+1}")
                        break

            fold_losses.append(best_val_loss)

        return np.mean(fold_losses)

    # Create an Optuna study and optimize the hyperparameters
    study = optuna.create_study(direction='minimize')
    study.optimize(objective, n_trials=30)

    # Print the best hyperparameters and best value
    print("Best hyperparameters:", study.best_params)
    print("Best value:", study.best_value)

    # Train the final model with the best hyperparameters and early stopping
    best_learning_rate = study.best_params['learning_rate']
    best_lasso_coef = study.best_params['lasso_coef']
    best_patience = study.best_params['patience']

    model = LogisticRegression(input_dim, output_dim, best_lasso_coef).to(device)
    optimizer = optim.Adam(model.parameters(), lr=best_learning_rate)
    criterion = nn.BCELoss()

    train_dataset = TensorDataset(X_train_val, y_train_val)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    best_train_loss = float('inf')
    counter = 0

    for epoch in range(num_epochs):
        train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)

            outputs = model(batch_X)
            loss = criterion(outputs, batch_y) + model.lasso_loss()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)
        print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}")

        if train_loss < best_train_loss:
            best_train_loss = train_loss
            counter = 0
        else:
            counter += 1

        if counter >= best_patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

    # Save the final model
    model_save_path = chr_model_folder + f'final_model_chr{chromosome_number}.pth'
    torch.save(model.state_dict(), model_save_path)
    print(f"Final model saved at: {model_save_path}")

    # Evaluate the final model on the test set
    with torch.no_grad():
        test_outputs = model(X_test.to(device))
        test_preds = (test_outputs > 0.5).float()
        test_accuracy = float(((test_preds > 0.5) == y_test).float().mean())
        test_precision = precision_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_recall = recall_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_f1 = f1_score(y_test.cpu().numpy(), test_preds.cpu().numpy(), average='micro')
        test_roc_auc = roc_auc_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), average='micro')
        test_r2 = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy())

        # Calculate false positive rate
        cm = confusion_matrix(y_test.cpu().numpy().ravel(), test_preds.cpu().numpy().ravel())
        tn, fp, fn, tp = cm.ravel()
        test_fpr = fp / (fp + tn)

        # Append performance metrics to the lists
        accuracies.append(test_accuracy)
        precisions.append(test_precision)
        recalls.append(test_recall)
        false_positive_rates.append(test_fpr)
        auc_rocs.append(test_roc_auc)
        r2_scores.append(test_r2)

        # Calculate individual R^2 scores for each SNP
        individual_r2_scores = sklearn_r2_score(y_test.cpu().numpy(), test_outputs.cpu().numpy(), multioutput='raw_values')

        # Get the names of the SNPs from the original dataframe
        snp_names = data.filter(regex='Unknown').columns

        # Save individual R^2 scores to a CSV file
        csv_file = chr_csv_folder + f'individual_r2_scores_chr{chromosome_number}.csv'

        with open(csv_file, 'w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['SNP', 'R2 Score'])
            for snp, r2_score in zip(snp_names, individual_r2_scores):
                writer.writerow([snp, r2_score])

        print(f"Individual R^2 scores saved at: {csv_file}")

        # Save individual AUC ROC curves for each SNP
        for i, snp in enumerate(snp_names):
            try: 
                fpr, tpr, _ = roc_curve(y_test.cpu().numpy()[:, i], test_outputs.cpu().numpy()[:, i])
                plt.figure()
                plt.plot(fpr, tpr, label=f'AUC ROC = {roc_auc_score(y_test.cpu().numpy()[:, i], test_outputs.cpu().numpy()[:, i]):.4f}')
                plt.xlabel('False Positive Rate')
                plt.ylabel('True Positive Rate')
                plt.title(f'AUC ROC Curve - {snp}')
                plt.legend()
                
                curve_file = chr_curve_folder + f'auc_roc_curve_{snp}_chr{chromosome_number}.png'
                plt.savefig(curve_file)
                plt.close()
            except ValueError:
                # Save a placeholder image if there is insufficient data
                plt.figure()
                plt.axis('off')
                plt.text(0.5, 0.5, "Insufficient data for ROC curve", ha='center', va='center')
                curve_file = chr_curve_folder + f'auc_roc_curve_{snp}_chr{chromosome_number}.png'
                plt.savefig(curve_file)
                plt.close()

                print(f"Skipping SNP {snp} due to insufficient data")


        print(f"Individual AUC ROC curves saved in: {curve_folder}")

        # Create a DataFrame to store the performance metrics for each chromosome
        performance_df = pd.DataFrame({
            'Chromosome': list(range(start, chromosome_number + 1)),
            'Accuracy': accuracies,
            'Precision': precisions,
            'Recall': recalls,
            'False Positive Rate': false_positive_rates,
            'AUC ROC': auc_rocs,
            'R2 Score': r2_scores
        })

        # Save the performance metrics to a CSV file
        performance_csv_file = csv_folder + 'performance_metrics.csv'
        performance_df.to_csv(performance_csv_file, index=False)
        print(f"Performance metrics saved at: {performance_csv_file}")

[I 2024-04-25 16:42:08,178] A new study created in memory with name: no-name-bd0cc460-1c7e-4394-b13e-7fc03a509ebb


Unknown PRS313 SNPs:  18
Known PRS313 SNPs:  4
23AndMe SNPs with LD to Unknown PRS313 SNPs:  496
Early stopping at epoch 211
Early stopping at epoch 37
Early stopping at epoch 18


[I 2024-04-25 16:42:16,971] Trial 0 finished with value: 0.29145585298538207 and parameters: {'learning_rate': 0.0007488122413980146, 'lasso_coef': 0.006881266906688167, 'patience': 17}. Best is trial 0 with value: 0.29145585298538207.


Early stopping at epoch 34
Early stopping at epoch 14
Early stopping at epoch 14


[I 2024-04-25 16:42:22,910] Trial 1 finished with value: 0.05943194907158613 and parameters: {'learning_rate': 0.0013365013610442463, 'lasso_coef': 7.860825238863613e-05, 'patience': 13}. Best is trial 1 with value: 0.05943194907158613.


...

[I 2024-04-25 16:55:24,254] Trial 29 finished with value: 0.15745203122496604 and parameters: {'learning_rate': 0.026921853285385215, 'lasso_coef': 0.00011281836327027214, 'patience': 5}. Best is trial 25 with value: 0.10184641219675541.


Early stopping at epoch 8
Early stopping at epoch 6
Best hyperparameters: {'learning_rate': 0.049203396615788146, 'lasso_coef': 1.0119257827823962e-05, 'patience': 7}
Best value: 0.10184641219675541
Epoch [1/500], Train Loss: 0.7135
Epoch [2/500], Train Loss: 0.3198
Epoch [3/500], Train Loss: 0.2397
Epoch [4/500], Train Loss: 0.2110
Epoch [5/500], Train Loss: 0.1908
Epoch [6/500], Train Loss: 0.1744
Epoch [7/500], Train Loss: 0.1679
Epoch [8/500], Train Loss: 0.1590
Epoch [9/500], Train Loss: 0.1552
Epoch [10/500], Train Loss: 0.1512
Epoch [11/500], Train Loss: 0.1493
Epoch [12/500], Train Loss: 0.1465
Epoch [13/500], Train Loss: 0.1450
Epoch [14/500], Train Loss: 0.1444
Epoch [15/500], Train Loss: 0.1398
Epoch [16/500], Train Loss: 0.1429
Epoch [17/500], Train Loss: 0.1466
Epoch [18/500], Train Loss: 0.1430
Epoch [19/500], Train Loss: 0.1263
Epoch [20/500], Train Loss: 0.1093
Epoch [21/500], Train Loss: 0.0993
Epoch [22/500], Train Loss: 0.0961
Epoch [23/500], Train Loss: 0.0978
Epoch

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/metrics/_ranking.py:1133: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(


<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>

In [ ]:
import numpy as np

def imputation_quality_score(imputed_data, true_data):
    """
    Calculates the Imputation Quality Score (IQS) between imputed data and true data.
    
    Args:
        imputed_data (numpy.ndarray): The imputed data matrix.
        true_data (numpy.ndarray): The true data matrix (with missing values).
        
    Returns:
        float: The Imputation Quality Score (IQS).
    """
    # Get the indices of missing values in the true data
    missing_indices = np.isnan(true_data)
    
    # Extract the imputed values and true values at the missing indices
    imputed_values = imputed_data[missing_indices]
    true_values = true_data[missing_indices]
    
    # Calculate the sum of squared differences between imputed and true values
    squared_diff_imputed = np.sum((imputed_values - true_values) ** 2)
    
    # Calculate the mean of the true values
    mean_true = np.nanmean(true_data)
    
    # Calculate the sum of squared differences between mean and true values
    squared_diff_mean = np.sum((mean_true - true_values) ** 2)
    
    # Calculate the Imputation Quality Score (IQS)
    iqs = 1 - (squared_diff_imputed / squared_diff_mean)
    
    return iqs